# Lab 4 – Image Segmentation (Updated with Loop for All Images)
This notebook processes all six Lab 4 images automatically.

In [ ]:
import cv2, numpy as np, glob
import matplotlib.pyplot as plt

# Load all images from folder
image_paths = glob.glob('lab4_pics/*')
print('Images found:', image_paths)

## Prewitt, Sobel, Canny, Hough, Otsu, Adaptive (processed for each image)

In [ ]:
for path in image_paths:
    print('\nProcessing:', path)
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)

    # Prewitt
    Gx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
    Gy = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    px = cv2.filter2D(blur,-1,Gx)
    py = cv2.filter2D(blur,-1,Gy)
    prewitt = cv2.magnitude(px.astype(float), py.astype(float))

    # Sobel
    sx = cv2.Sobel(blur, cv2.CV_64F,1,0,ksize=3)
    sy = cv2.Sobel(blur, cv2.CV_64F,0,1,ksize=3)
    sobel = cv2.magnitude(sx, sy)

    # Canny
    canny = cv2.Canny(blur,50,150)

    # Hough
    edges = cv2.Canny(blur,50,150)
    lines = cv2.HoughLines(edges,1,np.pi/180,120)
    draw = img.copy()
    if lines is not None:
        for r,theta in lines[:,0]:
            a,b = np.cos(theta), np.sin(theta)
            x0,y0 = a*r, b*r
            x1,y1 = int(x0+1000*(-b)), int(y0+1000*(a))
            x2,y2 = int(x0-1000*(-b)), int(y0-1000*(a))
            cv2.line(draw,(x1,y1),(x2,y2),(0,0,255),2)

    # Otsu
    ret, otsu = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Adaptive
    adapt = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY,11,2)

    # Display
    plt.figure(figsize=(12,8))
    plt.suptitle(path, fontsize=14)

    plt.subplot(231); plt.imshow(prewitt,cmap='gray'); plt.title('Prewitt')
    plt.subplot(232); plt.imshow(sobel,cmap='gray'); plt.title('Sobel')
    plt.subplot(233); plt.imshow(canny,cmap='gray'); plt.title('Canny')
    plt.subplot(234); plt.imshow(cv2.cvtColor(draw,cv2.COLOR_BGR2RGB)); plt.title('Hough')
    plt.subplot(235); plt.imshow(otsu,cmap='gray'); plt.title('Otsu')
    plt.subplot(236); plt.imshow(adapt,cmap='gray'); plt.title('Adaptive')

    plt.tight_layout()
    plt.show()
